## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Inhambane,MZ,-23.8650,35.3833,75.18,100,75,6.91,drizzle
1,1,Ushuaia,AR,-54.8000,-68.3000,41.47,91,100,4.34,overcast clouds
2,2,Grindavik,IS,63.8424,-22.4338,30.67,74,40,19.57,scattered clouds
3,3,Lüderitz,NaN,-26.6481,15.1594,69.22,78,46,3.11,scattered clouds
4,4,Kashgar,CN,39.4547,75.9797,28.09,62,100,6.38,overcast clouds
5,5,Yellowknife,CA,62.4560,-114.3525,-17.88,69,0,9.22,clear sky
6,6,Minsk,BY,53.9000,27.5667,31.75,93,100,10.83,overcast clouds
7,7,Arlit,NE,18.7369,7.3853,65.43,15,0,10.11,clear sky
8,8,Taltal,CL,-25.4000,-70.4833,65.39,84,18,1.28,few clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,74.55,94,62,3.00,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Inhambane,MZ,-23.8650,35.3833,75.18,100,75,6.91,drizzle
10,10,Vaini,TO,-21.2000,-175.2000,77.36,92,100,33.13,heavy intensity rain
14,14,Tual,ID,-5.6667,132.7500,83.93,71,100,6.35,light rain
17,17,Atuona,PF,-9.8000,-139.0333,78.98,77,3,22.88,clear sky
24,24,Bukama,CD,-9.2000,25.8500,77.72,78,80,1.07,broken clouds
25,25,Eyl,SO,7.9803,49.8164,82.45,50,7,13.73,clear sky
28,28,Avarua,CK,-21.2078,-159.7750,78.85,78,75,8.05,broken clouds
30,30,Buala,SB,-8.1450,159.5921,81.73,83,51,7.96,light rain
33,33,Hithadhoo,MV,-0.6000,73.0833,82.72,72,9,10.92,clear sky
35,35,Mossamedes,AO,-15.1961,12.1522,76.87,53,90,6.40,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
len(preferred_cities_df)

159

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df_nan = preferred_cities_df.dropna()
len(preferred_cities_df_nan)

158

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df_nan[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Inhambane,MZ,75.18,drizzle,-23.8650,35.3833,
10,Vaini,TO,77.36,heavy intensity rain,-21.2000,-175.2000,
14,Tual,ID,83.93,light rain,-5.6667,132.7500,
17,Atuona,PF,78.98,clear sky,-9.8000,-139.0333,
24,Bukama,CD,77.72,broken clouds,-9.2000,25.8500,
25,Eyl,SO,82.45,clear sky,7.9803,49.8164,
28,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
30,Buala,SB,81.73,light rain,-8.1450,159.5921,
33,Hithadhoo,MV,82.72,clear sky,-0.6000,73.0833,
35,Mossamedes,AO,76.87,overcast clouds,-15.1961,12.1522,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

    
# 6b. Iterate through the hotel DataFrame.   
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Inhambane,MZ,75.18,drizzle,-23.8650,35.3833,Oceano
10,Vaini,TO,77.36,heavy intensity rain,-21.2000,-175.2000,Keleti Beach Resort
14,Tual,ID,83.93,light rain,-5.6667,132.7500,Grand Vilia Hotel
17,Atuona,PF,78.98,clear sky,-9.8000,-139.0333,Villa Enata
25,Eyl,SO,82.45,clear sky,7.9803,49.8164,Amiin Hotel
28,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,Paradise Inn
30,Buala,SB,81.73,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
33,Hithadhoo,MV,82.72,clear sky,-0.6000,73.0833,Scoop Guest House
35,Mossamedes,AO,76.87,overcast clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
38,East London,ZA,76.77,light rain,-33.0153,27.9116,Tu Casa


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} °F</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))